## Benchmark Charts + Analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json

def load_data(filepath):
    try:
        with open(filepath) as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None

def preprocess_data(data, model):
    frameworks = ['ezkl', 'riscZero'] if model == 'random_forests' else ['ezkl', 'orion', 'riscZero']
    memory_usage = {framework: np.mean([int(mu.replace('kb', '')) for mu in data[model][framework]['memoryUsage']])
                    for framework in frameworks}
    proving_time = {framework: np.mean([float(pt.replace('s', '')) for pt in data[model][framework]['provingTime']])
                    for framework in frameworks}
    return memory_usage, proving_time

def plot_memory_usage(frameworks, memory_usage, title):
    valid_frameworks = [fw for fw in frameworks if fw in memory_usage]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(np.arange(len(valid_frameworks)), [memory_usage[fw] for fw in valid_frameworks], color='b', width=0.4, align='center')
    plt.xlabel('Frameworks')
    plt.ylabel('Average Memory Usage (kb)', color='b')
    plt.xticks(np.arange(len(valid_frameworks)), valid_frameworks)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}', ha='center', va='bottom', color='black')

    plt.title(title)
    plt.show()

def plot_proving_time(frameworks, proving_time, title):
    valid_frameworks = [fw for fw in frameworks if fw in proving_time]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(np.arange(len(valid_frameworks)), [proving_time[fw] for fw in valid_frameworks], color='r', width=0.4, align='center')
    plt.xlabel('Frameworks')
    plt.ylabel('Average Proving Time (s)', color='r')
    plt.xticks(np.arange(len(valid_frameworks)), valid_frameworks)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}', ha='center', va='bottom', color='black')

    plt.title(title)
    plt.show()


def calculate_performance(memory_usage, proving_time):
    speedup_risczero = proving_time['riscZero'] / proving_time['ezkl']
    reduction_risczero = (memory_usage['riscZero'] - memory_usage['ezkl']) * 100 / memory_usage['riscZero']

    if 'orion' in memory_usage:
        speedup_orion = proving_time['orion'] / proving_time['ezkl']
        reduction_orion = (memory_usage['orion'] - memory_usage['ezkl']) * 100 / memory_usage['orion']
        return speedup_orion, reduction_orion, speedup_risczero, reduction_risczero
    else:
        return None, None, speedup_risczero, reduction_risczero

def print_performance(model, performance):
    speedup_orion, reduction_orion, speedup_risczero, reduction_risczero = performance
    statement = f"{model} Results:\n\nProving Time Speedup:\n"
    if speedup_orion is not None:
        statement += f"EZKL is approximately {speedup_orion:.2f} times faster than Orion.\n"
    statement += f"EZKL is about {speedup_risczero:.2f} times faster than RISC0.\n\nMemory Usage Reduction:\n"
    if reduction_orion is not None:
        statement += f"EZKL uses roughly {reduction_orion:.2f}% less memory compared to Orion.\n"
    statement += f"EZKL's memory usage is about {reduction_risczero:.2f}% less than that of RISC0.\n"
    print(statement)

# Main Execution
data = load_data('../benchmarks.json')
if data:
    models = ['linear_regressions', 'random_forests', 'svm_classifications', 'te_regressions']
    frameworks = ['ezkl', 'orion', 'riscZero']
    total_memory_usage = {framework: 0 for framework in frameworks}
    total_proving_time = {framework: 0 for framework in frameworks}

    for model in models:
        memory_usage, proving_time = preprocess_data(data, model)
        plot_memory_usage(frameworks, memory_usage, f'{model}: Memory Usage')
        plot_proving_time(frameworks, proving_time, f'{model}: Proving Time')
        performance = calculate_performance(memory_usage, proving_time)
        print_performance(model, performance)
        # skip random forests for mean calculations since it omits orion
        if model == 'random_forests':
            continue
        for framework in frameworks:
            total_memory_usage[framework] += memory_usage.get(framework, 0)
            total_proving_time[framework] += proving_time.get(framework, 0)

    mean_memory_usage = {fw: mu / (len(models)-1) for fw, mu in total_memory_usage.items()}
    mean_proving_time = {fw: pt / (len(models)-1) for fw, pt in total_proving_time.items()}
    plot_memory_usage(frameworks, mean_memory_usage, 'Mean Memory Usage Across All Models (except random forest)')
    plot_proving_time(frameworks, mean_proving_time, 'Mean Proving Time Across All Models (except random forest)')
    performance = calculate_performance(mean_memory_usage, mean_proving_time)
    print_performance("Mean", performance)